In [3]:
# Import statements
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.transforms import Bbox
from matplotlib.figure import Figure
from matplotlib.colors import LinearSegmentedColormap
import sys
from glob import glob
from pymatreader import read_mat
from scipy import signal
from scipy import stats
from scipy.ndimage import uniform_filter1d

import itertools

#import matlab.engine
# Test out matlab spike detection code
#eng = matlab.engine.start_matlab()
#eng.eval("startup", nargout=0)

# Importing custom python files
import consts
import importlib
importlib.reload(consts)

from open_ephys.analysis import Session

%matplotlib tk

f = os.sep

In [4]:
# Read in pickle file
interm_data_path =  f + 'home' +f+ 'pierfier' +f+ 'Projects' +f+ 'Pierre Fabris' +f+ 'PV DBS neocortex' +f+ 'Interm_Data' +f+ 'flicker.pkl'
savefig_path = f + 'home' +f+ 'pierfier' +f+ 'Dropbox' +f+ 'RKC-HanLab' +f+ 'Pierre PV DBS Project Dropbox' +f+ 'Materials' +f+ 'Plots' +f+ 'Flicker' +f+ 'Compact' +f
df =pd.read_pickle(interm_data_path)
print(df.columns)

Index(['frame_time', 'raw_trace', 'sub_vm', 'detrend_vm', 'trace_noise',
       'interp_time', 'interp_raw_trace', 'interp_raw_detrend_trace',
       'interp_spike_raster', 'spike_amp_raster', 'flicker_raster',
       'stim_raster', 'interp_subvm', 'stim_freq', 'flicker_freq', 'mouse_id',
       'session_id', 'trial_id', 'stim_param', 'fov_id', 'roi_id'],
      dtype='object')


In [5]:
# Loop through each ROI
test_freqs = df['stim_freq'].unique()
for freq in test_freqs:
    stim_df = df[df['stim_freq'] == freq]
    
    mouse_list = stim_df['mouse_id'].unique()
    session_list = stim_df['session_id'].unique()
    fov_list = stim_df['fov_id'].unique()

    pairings = list(itertools.product(mouse_list, session_list, fov_list))
    
    # Store all of the neurons trial-averaged vm
    vm_df = pd.DataFrame()

    # Loop through unique FOVs
    roi_acum = 0
    for values in pairings:
        fov_df = stim_df[(stim_df['mouse_id'] == values[0]) & \
                         (stim_df['session_id'] == values[1]) & \
                            (stim_df['fov_id'] == values[2])]
        
        # Skip if dataframe is empty()
        if fov_df.empty:
            continue
        flicker_start = fov_df[fov_df['flicker_raster'] == 1]['interp_time'].values[0]
        # Timeline for all
        timeline = fov_df[fov_df['trial_id'] == fov_df['trial_id'].unique()[0]]['interp_time'].values - flicker_start
        
        #Calculate each trials normalized spike amplitude and set it as a new column
        # in the FOV dataframe
        for tr_val in fov_df['trial_id'].unique():
            trial_df = fov_df[fov_df['trial_id'] == tr_val]
            avg_sp_amp = np.nanmean(trial_df['spike_amp_raster'].values)
            fov_df.loc[fov_df['trial_id'] == tr_val, 'interp_norm_vm'] = trial_df['interp_subvm'].values/avg_sp_amp
            
        avg_trial = fov_df.groupby('interp_time')['interp_norm_vm'].mean()

        nr_dict = {
            'neuron':roi_acum,
            'vm_trial_avg':avg_trial.values,
            'interp_time':timeline
        }
        vm_df = pd.concat([vm_df, pd.DataFrame(nr_dict)], ignore_index=True, join='outer')
        vm_df['neuron'] = vm_df['neuron'].astype('category')
        roi_acum += 1
    
    # Plot the trial averaged Vm
    plot_df = vm_df.pivot(index='neuron', columns='interp_time', values='vm_trial_avg')
    avg_vm = plot_df.mean(axis=0)
    std_vm = plot_df.std(axis=0)
    num_neurons = plot_df.shape[0]
    sem_vm = std_vm/np.sqrt(num_neurons)

    cm = 1/2.54
    plt.figure(figsize=(10.5 * cm, 3.5 * cm))
    plt.fill(np.concatenate((timeline, timeline[::-1])), \
            np.concatenate((avg_vm + sem_vm, avg_vm[::-1] - sem_vm[::-1])),
            color='gray')
    plt.plot(timeline, avg_vm, '-k', linewidth=0.75)
    
    # Plot the stimultion protocol
    plt.plot(timeline, 0.07*trial_df['flicker_raster'] + .10 + np.max(avg_vm + sem_vm), '-b')
    plt.plot([timeline[0], timeline[-1]], np.ones((2))* (.01 + np.max(avg_vm + sem_vm)), color=consts.pulse_color)
    stim_idx = np.where(trial_df['stim_raster'].values == 1)[0]
    plt.plot(timeline[stim_idx], np.ones_like(stim_idx)*(.01 + np.max(avg_vm + sem_vm)), '|',\
         linewidth=20,  color=consts.pulse_color)
    plt.xlabel('Time from flicker onset (s)')
    plt.ylabel('Normalized Vm')
    plt.title('Flicker Average: ' + str(freq))
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    save_filename = savefig_path + 'Population_Vm_' +\
                    str(stim_df['stim_freq'].unique()[0]) + 'Hz'

    plt.savefig(save_filename + '.svg', format='svg')
    plt.savefig(save_filename + '.png', format='png')
    
    plt.show()

/tmp/ipykernel_118493/3914309815.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fov_df.loc[fov_df['trial_id'] == tr_val, 'interp_norm_vm'] = trial_df['interp_subvm'].values/avg_sp_amp
/tmp/ipykernel_118493/3914309815.py:33: RuntimeWarning: Mean of empty slice
  avg_sp_amp = np.nanmean(trial_df['spike_amp_raster'].values)
/tmp/ipykernel_118493/3914309815.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fov_df.loc[fov_df['trial_id'] == tr_val, 'interp_norm_vm'] = trial_df['interp_subvm'].values/avg_

In [16]:
# Plot both the low pass filtered population Vm and high pass filtered Vm
# Loop through each ROI
cutoff_freq = [3.5, 4.5] # hz
samp_freq =  500 # Hz
sos_stop = signal.butter(N=2, Wn=cutoff_freq, btype='bandstop' , fs=samp_freq, output='sos')
sos_pass = signal.butter(N=2, Wn=cutoff_freq, btype='bandpass' , fs=samp_freq, output='sos')

test_freqs = df['stim_freq'].unique()
for freq in test_freqs:
    stim_df = df[df['stim_freq'] == freq]
    
    mouse_list = stim_df['mouse_id'].unique()
    session_list = stim_df['session_id'].unique()
    fov_list = stim_df['fov_id'].unique()

    pairings = list(itertools.product(mouse_list, session_list, fov_list))
    
    # Store all of the neurons trial-averaged vm
    vm_df = pd.DataFrame()

    vm_stop_4Hz_df = pd.DataFrame()
    vm_pass_4Hz_df = pd.DataFrame()
    
    # Loop through unique FOVs
    roi_acum = 0
    for values in pairings:
              
        # Individual FOV bandstop filtered Vm
        fov_bandstop_df = stim_df[(stim_df['mouse_id'] == values[0]) & \
                         (stim_df['session_id'] == values[1]) & \
                            (stim_df['fov_id'] == values[2])]
        
        # Individual FOV highpass filtered Vm
        fov_bandpass_df = stim_df[(stim_df['mouse_id'] == values[0]) & \
                         (stim_df['session_id'] == values[1]) & \
                            (stim_df['fov_id'] == values[2])]
        
        # Skip if dataframe is empty()
        if fov_bandstop_df.empty:
            continue
        flicker_start = fov_bandstop_df[fov_bandstop_df['flicker_raster'] == 1]['interp_time'].values[0]
        # Timeline for all
        timeline = fov_bandstop_df[fov_bandstop_df['trial_id'] == fov_bandstop_df['trial_id'].unique()[0]]['interp_time'].values - flicker_start
        
        #Calculate each trials normalized spike amplitude and set it as a new column
        # in the FOV dataframe
        for tr_val in fov_bandstop_df['trial_id'].unique():
            trial_df = fov_bandstop_df[fov_bandstop_df['trial_id'] == tr_val]
            avg_sp_amp = np.nanmean(trial_df['spike_amp_raster'].values)
            fov_bandstop_df.loc[fov_bandstop_df['trial_id'] == tr_val, 'interp_norm_vm'] = signal.sosfiltfilt(sos_stop, trial_df['interp_subvm'].values/avg_sp_amp)
            fov_bandpass_df.loc[fov_bandstop_df['trial_id'] == tr_val, 'interp_norm_vm'] = signal.sosfiltfilt(sos_pass, trial_df['interp_subvm'].values/avg_sp_amp)

            timeline = np.arange(np.array(trial_df['interp_subvm'].values/avg_sp_amp).shape[0])

            # DEBUG
            #fig, axs = plt.subplots(2, sharex=True)
            #axs[0].plot(timeline, fov_bandstop_df.loc[fov_bandstop_df['trial_id'] == tr_val, 'interp_norm_vm'], color='r', label='Band Stop Filtered')
            #axs[0].plot(timeline, trial_df['interp_subvm'].values/avg_sp_amp, color='b', label='Original Trace')
            #axs[0].legend()
#
            #axs[0].set_title('Bandstop Filtered Trace Overlay')
#
            
#
            #axs[1].plot(timeline, fov_bandpass_df.loc[fov_bandstop_df['trial_id'] == tr_val, 'interp_norm_vm'], color='r', label='Band Pass Filtered')
            #axs[1].plot(timeline, trial_df['interp_subvm'].values/avg_sp_amp, color='b', label='Original Trace')
            #axs[1].legend()
            #axs[1].set_title('Bandpass Filtered Trace Overlay')
#
            #plt.show()
            
        avg_stop_trial = fov_bandstop_df.groupby('interp_time')['interp_norm_vm'].mean()
        avg_pass_trial = fov_bandpass_df.groupby('interp_time')['interp_norm_vm'].mean()

        nr_dict = {
            'neuron':roi_acum,
            'vm_trial_avg':avg_stop_trial.values,
            'interp_time':timeline
        }

        vm_stop_4Hz_df = pd.concat([vm_stop_4Hz_df, pd.DataFrame(nr_dict)], ignore_index=True, join='outer')
        vm_stop_4Hz_df['neuron'] = vm_stop_4Hz_df['neuron'].astype('category')
        
        nr_dict = {
            'neuron':roi_acum,
            'vm_trial_avg':avg_pass_trial.values,
            'interp_time':timeline
        }

        vm_pass_4Hz_df = pd.concat([vm_pass_4Hz_df, pd.DataFrame(nr_dict)], ignore_index=True, join='outer')
        vm_pass_4Hz_df['neuron'] = vm_pass_4Hz_df['neuron'].astype('category')
        
        roi_acum += 1
    
    # Plot the trial averaged Vm bandstop
    plot_df = vm_stop_4Hz_df.pivot(index='neuron', columns='interp_time', values='vm_trial_avg')
    avg_vm = plot_df.mean(axis=0)
    std_vm = plot_df.std(axis=0)
    num_neurons = plot_df.shape[0]
    sem_vm = std_vm/np.sqrt(num_neurons)

    cm = 1/2.54
    plt.figure(figsize=(25 * cm, 10 * cm))
    plt.fill(np.concatenate((timeline, timeline[::-1])), \
            np.concatenate((avg_vm + sem_vm, avg_vm[::-1] - sem_vm[::-1])),
            color='gray')
    plt.plot(timeline, avg_vm, '-k', linewidth=0.75)
    
    # Plot the stimultion protocol
    plt.plot(timeline, 0.07*trial_df['flicker_raster'] + .10 + np.max(avg_vm + sem_vm), '-b')
    plt.plot([timeline[0], timeline[-1]], np.ones((2))* (.01 + np.max(avg_vm + sem_vm)), color=consts.pulse_color)
    stim_idx = np.where(trial_df['stim_raster'].values == 1)[0]
    plt.plot(timeline[stim_idx], np.ones_like(stim_idx)*(.01 + np.max(avg_vm + sem_vm)), '|',\
         linewidth=20,  color=consts.pulse_color)
    plt.xlabel('Time from flicker onset (s)')
    plt.ylabel('Normalized Vm')
    plt.title('Flicker Average with 4 Hz Removal: ' + str(freq))
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    save_filename = savefig_path + 'Population_Vm_bandstop_4Hz_' +\
                    str(stim_df['stim_freq'].unique()[0]) + 'Hz'

    plt.savefig(save_filename + '.svg', format='svg')
    plt.savefig(save_filename + '.png', format='png')

    # Plot the trial averaged Vm bandpass
    plot_df = vm_pass_4Hz_df.pivot(index='neuron', columns='interp_time', values='vm_trial_avg')
    avg_vm = plot_df.mean(axis=0)
    std_vm = plot_df.std(axis=0)
    num_neurons = plot_df.shape[0]
    sem_vm = std_vm/np.sqrt(num_neurons)

    cm = 1/2.54
    plt.figure(figsize=(25 * cm, 10 * cm))
    plt.fill(np.concatenate((timeline, timeline[::-1])), \
            np.concatenate((avg_vm + sem_vm, avg_vm[::-1] - sem_vm[::-1])),
            color='gray')
    plt.plot(timeline, avg_vm, '-k', linewidth=0.75)
    
    # Plot the stimultion protocol
    plt.plot(timeline, 0.07*trial_df['flicker_raster'] + .10 + np.max(avg_vm + sem_vm), '-b')
    plt.plot([timeline[0], timeline[-1]], np.ones((2))* (.01 + np.max(avg_vm + sem_vm)), color=consts.pulse_color)
    stim_idx = np.where(trial_df['stim_raster'].values == 1)[0]
    plt.plot(timeline[stim_idx], np.ones_like(stim_idx)*(.01 + np.max(avg_vm + sem_vm)), '|',\
         linewidth=20,  color=consts.pulse_color)
    plt.xlabel('Time from flicker onset (s)')
    plt.ylabel('Normalized Vm')
    plt.title('Flicker Average of just 4 Hz: ' + str(freq))
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    save_filename = savefig_path + 'Population_Vm_bandpass_4Hz_' +\
                    str(stim_df['stim_freq'].unique()[0]) + 'Hz'

    plt.savefig(save_filename + '.svg', format='svg')
    plt.savefig(save_filename + '.png', format='png')
    
plt.show()

/tmp/ipykernel_118493/18111135.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fov_bandstop_df.loc[fov_bandstop_df['trial_id'] == tr_val, 'interp_norm_vm'] = signal.sosfiltfilt(sos_stop, trial_df['interp_subvm'].values/avg_sp_amp)
/tmp/ipykernel_118493/18111135.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fov_bandpass_df.loc[fov_bandstop_df['trial_id'] == tr_val, 'interp_norm_vm'] = signal.sosfiltfilt(sos_pass, trial_df['interp_subvm'].values/avg_sp_amp)
/tmp/ipykernel_118493/18111135.py:49: Ru

In [ ]:
# Calculate the firing rate for each neuron
spike_rate_win = 50
samp_freq = 500
test_freqs = df['stim_freq'].unique()

for freq in test_freqs:
    stim_df = df[df['stim_freq'] == freq]
    
    mouse_list = stim_df['mouse_id'].unique()
    session_list = stim_df['session_id'].unique()
    fov_list = stim_df['fov_id'].unique()

    pairings = list(itertools.product(mouse_list, session_list, fov_list))
    
    # Store all of the neurons trial-averaged vm
    fr_df = pd.DataFrame()

    # Loop through unique FOVs
    roi_acum = 0
    for values in pairings:
        fov_df = stim_df[(stim_df['mouse_id'] == values[0]) & \
                         (stim_df['session_id'] == values[1]) & \
                            (stim_df['fov_id'] == values[2])]
        
        # Skip if dataframe is empty()
        if fov_df.empty:
            continue
        flicker_start = fov_df[fov_df['flicker_raster'] == 1]['interp_time'].values[0]
        # Timeline for all
        timeline = fov_df[fov_df['trial_id'] == fov_df['trial_id'].unique()[0]]['interp_time'].values - flicker_start
        
        #Calculate each trials firing rate and set it as a new column
        # in the FOV dataframe
        for tr_val in fov_df['trial_id'].unique():
            trial_df = fov_df[fov_df['trial_id'] == tr_val]
            
            firing_rate = uniform_filter1d(trial_df['interp_spike_raster'].values*samp_freq, size=25)

            #TODO save the firing rate instead of this Vm stuff
            fov_df.loc[fov_df['trial_id'] == tr_val, 'firing_rate'] = firing_rate

        avg_trial = fov_df.groupby('interp_time')['firing_rate'].mean()

        nr_dict = {
            'neuron':roi_acum,
            'fr_trial_avg':avg_trial.values,
            'interp_time':timeline
        }
        fr_df = pd.concat([fr_df, pd.DataFrame(nr_dict)], ignore_index=True, join='outer')
        fr_df['neuron'] = fr_df['neuron'].astype('category')
        roi_acum += 1
    
    # Plot the trial averaged Vm
    plot_df = fr_df.pivot(index='neuron', columns='interp_time', values='fr_trial_avg')
    avg_fr = plot_df.mean(axis=0)
    std_fr = plot_df.std(axis=0)
    num_neurons = plot_df.shape[0]
    sem_fr = std_fr/np.sqrt(num_neurons)

    cm = 1/2.54
    plt.figure(figsize=(10.5 * cm, 3.5 * cm))
    plt.fill(np.concatenate((timeline, timeline[::-1])), \
            np.concatenate((avg_fr + sem_fr, avg_fr[::-1] - sem_fr[::-1])),
            color='gray')
    plt.plot(timeline, avg_fr, '-k')
    
    # Plot the stimultion protocol
    plt.plot(timeline, 1*trial_df['flicker_raster'] + 1 + np.max(avg_fr + sem_fr), '-b')
    plt.plot([timeline[0], timeline[-1]], np.ones((2))* (.01 + np.max(avg_fr + sem_fr)), color=consts.pulse_color)
    stim_idx = np.where(trial_df['stim_raster'].values == 1)[0]
    plt.plot(timeline[stim_idx], np.ones_like(stim_idx)*(.01 + np.max(avg_fr + sem_fr)), '|',\
         linewidth=20,  color=consts.pulse_color)
    plt.xlabel('Time from flicker onset (s)')
    plt.ylabel('Firing Rate (Hz)')
    plt.title('Flicker Average: ' + str(freq))
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    save_filename = savefig_path + 'Population_FR_' +\
                    str(stim_df['stim_freq'].unique()[0]) + 'Hz'

    plt.savefig(save_filename + '.svg', format='svg')
    plt.savefig(save_filename + '.png', format='png')
    
    plt.show()

In [14]:
plt.close('all')

In [ ]:
# check the padding
raster = np.zeros(150)
raster[(150*np.random.rand(30)).astype(int)] = 1

plt.figure()
plt.plot(raster, label='original')
padded_raster = np.pad(raster, (25, 25), mode='edge')

plt.plot(padded_raster + 0.5, label='padded')
mv_avg = uniform_filter1d(padded_raster, size=25, mode='constant')
plt.plot(mv_avg + 1, label='Firing Rate')
plt.legend()
plt.show()